<a href="https://colab.research.google.com/github/RafaRojasH/AI-Finanzas/blob/main/RandomForestAIF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Random Forest


Librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
plt.style.use('bmh')
from google.colab import files
from google.colab import drive

Cargar datos

In [69]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BA_05_07_2021_25_11_2021.csv')
df.set_index("Date", inplace=True)
df.dropna(inplace=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Obtener los datos del rendimiento
$$rendimiento = \frac{Precio_{actual}-Precio_{anterior}}{Precio_{anterior}}$$

In [72]:
rendimiento = []
rendimiento.append(0.0)
#df['Rendimiento'][0] = np.float64(0)
for i in range(1, len(df)):
    P_actual = df['Close'][i]
    P_anterior = df['Close'][i - 1]
    rendimiento.append(float((P_actual - P_anterior) / P_anterior))
df['Rendmiento'] = rendimiento

Preparar datos para entrada y salida

In [73]:
x = df.iloc[:, df.columns.size-1:df.columns.size].values
y = df.iloc[:, df.columns.size-1].values

Crear conjuntos de entrenamiento y prueba

In [74]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.26,  random_state=0)
scale = StandardScaler()
x_train = scale.fit_transform(x_train)
x_test = scale.transform(x_test)

Establecer parámetros para modelo de predicción, para encontrar el mejor modelo

In [75]:
grid_rf = {
'n_estimators': [20, 50, 100, 500, 1000],  
'max_depth': np.arange(1, 15, 1),  
'min_samples_split': [2, 10, 9], 
'min_samples_leaf': np.arange(1, 15, 2, dtype=int),  
'bootstrap': [True, False], 
'random_state': [1, 2, 30, 42]
}
rscv = RandomizedSearchCV(estimator=RandomForestRegressor(), param_distributions=grid_rf, cv=3, n_jobs=-1, verbose=2, n_iter=200)
rscv_fit = rscv.fit(x_train, y_train)
best_parameters = rscv_fit.best_params_
print(best_parameters)

Fitting 3 folds for each of 200 candidates, totalling 600 fits
{'random_state': 42, 'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 14, 'bootstrap': False}


Aplicar el modelo con los mejores parámetros y realizar el entrenamiento

In [76]:
modelRandomForest = RandomForestRegressor(n_estimators=best_parameters['n_estimators'], random_state=best_parameters['random_state'], 
                              min_samples_split=best_parameters['min_samples_split'], min_samples_leaf=best_parameters['min_samples_leaf'],
                              max_depth=best_parameters['max_depth'], bootstrap=best_parameters['bootstrap'])
modelRandomForest.fit(x_train, y_train)

RandomForestRegressor(bootstrap=False, max_depth=14, n_estimators=1000,
                      random_state=42)

Realizar la predicción con el conjunto de entrada de prueba

In [77]:
predict = modelRandomForest.predict(x_test)

Evaluar el rendimiento de la predicción

In [80]:
errors = abs(predict - y_test)
mape = 100 * (errors / y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.') 
print("Mean Absolute Error (MAE):", round(metrics.mean_absolute_error(y_test, predict), 4))
print("Mean Squared Error (MSE):", round(metrics.mean_squared_error(y_test, predict), 4))
print("Root Mean Squared Error (RMSE):", round(np.sqrt(metrics.mean_squared_error(y_test, predict)), 4))
print("R^2 Score:", round(metrics.r2_score(y_test, predict), 4))
print("Mean Absolute Percentage Error (MAPE):", round(np.mean(np.abs((y_test-predict) / y_test)) * 100))
print("Adjusted R^2 Score:", round(1 - (((1-np.mean(np.abs((y_test-predict) / y_test)) * 100)*(len(y_test)-1)/(len(y_test)-1-1)))))
#print(f'Train Score : {modelRandomForest.score(x_train, y_train) * 100:.2f}% and Test Score : {modelRandomForest.score(x_test, y_test) * 100:.2f}% using Random Tree Regressor.')

Accuracy: 99.33 %.
Mean Absolute Error (MAE): 0.0008
Mean Squared Error (MSE): 0.0
Root Mean Squared Error (RMSE): 0.0015
R^2 Score: 0.9948
Mean Absolute Percentage Error (MAPE): 7
Adjusted R^2 Score: 8
